# Python Notebook for "Mapping Green Skills in Collective Skill Formation Systems: A Natural Language Processing Analysis of Danish Vocational Education and Training"

## Content:

## 1. Loading documents and packages
### 1.1Divide documents into sections

## 2. Green dictionary search
### 2.1 Purpose
### 2.2. Competencies prior to admission
### 2.3. Competencies in the main course
### 2.4. Final exam

## 3. Add educational fields and date

## 4. Divide documents by competencies

## 5. Separate final exam from competencies

## 6. Classification and validation with LLM approach
### 6.1 Validation

## 7. Validation of LLM approach with manual review

## 8. Add final exams to dataset

## 9. Recode dataset - Rows are training ordinances instead of competencies



## 1. Loading documents and packages

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import pickle
import spacy
from openai import OpenAI
import tiktoken

In [ ]:
base_dir = r"(insert file location here)"

In [ ]:
data = []

In [ ]:
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # Check if the file is a PDF
        if file.lower().endswith('.pdf'):
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            try:
                # Open the PDF file
                with open(file_path, 'rb') as f:
                    reader = PdfReader(f)
                    text = ''
                    # Iterate through each page and extract text
                    for page_num in range(len(reader.pages)):
                        page = reader.pages[page_num]
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text
                    # Extract title from file name (without extension)
                    title = os.path.splitext(file)[0]
                    # Append the extracted text, title, and file path to the data list
                    data.append({
                        'Title': title,
                        'Content': text,
                        'File_Path': file_path  # New column for file path
                    })
            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

In [ ]:
df = pd.DataFrame(data, columns=['Title', 'Content', 'File_Path'])

In [ ]:
def extract_subfolder(file_path):
    # Get the directory containing the file
    dir_path = os.path.dirname(file_path)
    # Get the name of the directory (subfolder)
    subfolder = os.path.basename(dir_path)
    return subfolder

In [ ]:
df['Subfolder'] = df['File_Path'].apply(extract_subfolder)

In [ ]:
def extract_date(content):
    # Use a regular expression to find a date in the format 'DD/MM/YYYY'
    match = re.search(r"\b(\d{2}/\d{2}/\d{4})\b", content)
    if match:
        return match.group(1)  # Return the matched date
    else:
        return None 

In [ ]:
df['Date'] = df['Content'].apply(extract_date)

In [ ]:
def remove_footer(content):
    # Regular expression to match the footer structure
    footer_pattern = r"BEK nr \d{1,4} af \d{2}/\d{2}/\d{4} \d"
    # Use re.sub() to remove the matched footer from the content
    return re.sub(footer_pattern, '', content)

In [ ]:
df['Content'] = df['Content'].apply(remove_footer)

In [ ]:
df_e2015 = df[~df['File_Path'].str.contains("Kategoriserede uddannelser", na=False)]

In [ ]:
df_e2015.tail(10)

### 1.1 Divide documents into sections

In [ ]:
def extract_formål(content, Title):
    # Check if the title is "B20170033205" to apply the special rule
    if Title == "B20170033205":
        # Use regular expression for the specific rule for this title
        match = re.search(r"Formål og opdeling(.*?)Uddannelsen udbydes med talentspor.", content, re.DOTALL)
    else:
        # Original rule for other rows
        match = re.search(r"Formål og opdeling(.*?)Varighed", content, re.DOTALL)
    if match:
        return match.group(1).strip()  # Return the matched content (group 2), stripping any surrounding whitespace
    else:
        return None  # Return None if the pattern is not found

In [ ]:
df_e2015['formål_og_opdeling'] = df_e2015.apply(lambda row: extract_formål(row['Content'], row['Title']), axis=1)


In [ ]:
formål = df_e2015.loc[729, 'formål_og_opdeling'][:1000]
print(formål)

In [ ]:
def extract_kompetencer_forud(content, Title):
    # Check if the title is "B20170033205" to apply the special rule
    if Title == "B20170033205":
        # Use regular expression for the specific rule for this title
        match = re.search(r"(1, nr. 2, i lov om erhvervsuddannelser.)(.*?)(Kompetencer i hovedforløbet)", content, re.DOTALL)
    else:
        # Original rule for other rows
        match = re.search(r"(Kompetencer forud for optagelse på det studiekompetencegivende forløb og forløbets indhold|Kompetencer forud for optagelse til skoleundervisning i hovedforløbet|Kompetencer forud for optagelse til skoleundervisningen i hovedforløbet)(.*?)(Kompetencer i hovedforløbet|Kompetencer m\.v\. i hovedforløbet|Kompetencer mv\. i hovedforløbet)", content, re.DOTALL)
    
    if match:
        return match.group(2).strip()  # Return the matched content (group 2), stripping any surrounding whitespace
    else:
        return None  # Return None if the pattern is not found


In [ ]:
df_e2015['kompetencer_forud_optagelse'] = df_e2015.apply(lambda row: extract_kompetencer_forud(row['Content'], row['Title']), axis=1)


In [ ]:
def extract_kompetencer_hovedforløb(content):
    # Use regular expression to find text between "Kompetencer i hovedforløbet" or "Kompetencer m.v. i hovedforløbet" and "Godskrivning og merit"
    match = re.search(r"(Kompetencer i hovedforløbet|Kompetencer m\.v\. i hovedforløbet|Kompetencer mv\. i hovedforløbet)(.*?)(Godskrivning)", content, re.DOTALL)
    
    if match:
        return match.group(2).strip()  # Return the content between the matches, stripping any surrounding whitespace
    else:
        return None  # Return None if the pattern is not found

In [ ]:
df_e2015['kompetencer_hovedforløb'] = df_e2015['Content'].apply(extract_kompetencer_hovedforløb)

In [ ]:
def extract_afsluttende_prøve(content):
    # Regular expression to find text between the two sets of keywords
    match = re.search(r"(Afsluttende prøve og gymnasial eksamen|Afsluttende prøve og svendeprøve|Afsluttende prøve (svendeprøve)|Afsluttende prøve|Afsluttende prøve, svendeprøve og gymnasial eksamen|Svendeprøve og gymnasial eksamen (erhvervsfaglig studentereksamen)|Svendeprøve og gymnasial eksamen|Fagprøve og gymnasial eksamen|Fagprøve|Svendeprøve)(.*?)(Beviser|Ikrafttrædelse og overgangsbestemmelser)", content, re.DOTALL)
    
    if match:
        return match.group(2).strip()  # Return the content between the matches, stripping any surrounding whitespace
    else:
        return None

In [ ]:
def extract_afsluttende_prøve(content):
    # Use regular expression to find the text between the starting and ending phrases
    match = re.search(r"(Afsluttende prøve og gymnasial eksamen|Afsluttende prøve og svendeprøve|Afsluttende prøve \(svendeprøve\)|Afsluttende prøve|Afsluttende prøve, svendeprøve og gymnasial eksamen|Svendeprøve og gymnasial eksamen \(erhvervsfaglig studentereksamen\)|Svendeprøve og gymnasial eksamen|Fagprøve og gymnasial eksamen|Fagprøve|Svendeprøve)(.*?)(Beviser|Ikrafttrædelse og overgangsbestemmelser)", content, re.DOTALL)
    
    # Check if a match was found
    if match:
        return match.group(2).strip()  # Return the matched content, stripping any surrounding whitespace
    else:
        return None  # Return None if no match is found

In [ ]:
df_e2015['afsluttende_prøve'] = df_e2015['Content'].apply(extract_afsluttende_prøve)

In [ ]:
df_e2015.head()

In [ ]:
df_e2015.to_pickle('df_e2015_ny.pkl')


## 2. Green dictionary search

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('da_core_news_sm')

In [ ]:
word_list = ['luftkvalitet', 'akvakultur', 'bionedbryd', 'biodivers', 'biobrændstof', 'biomasse', 
             'co2', 'cirkulær', 'ren energi', 'klima', 'bevarelse', 'katastroferisikostyring', 
             'energieffekt', 'energihandel', 'miljø', 'geotermisk', 'gletsjerforsk', 'grøn', 
             'hydrologi', 'arealanvend', 'landskabsarkitekt', 'LEED', 'leadership in energy and environmental design',
             'naturlige pesticider', 'naturforsk', 
             'foruren', 'genbrug', 'vedvarende', 'solenergi', 'håndtering af affald', 'behandling af affald', 'affaldshåndt',
             'affaldsbehandl', 'affaldssort', 'bæredygtig', 'affaldsredu', 'spildevand', 
             'spildevandsreduktion', 'behandling af spildevand', 'reduktion af spildevand', 'vandressource', 
             'vejrsikring', 'dyreliv', 'vindenergi', 'udled', 'drivgas', 'ressourceeffekt', 
             'reduktion af material', 'vandforbrug', 'regenerativ', 'genanvend', 
             'giftige kemikalie', 'giftige stof', 'EPD', 'livscyklus', 'LCA', 'vedligehold', 'energioptim', 
             'ressourceanvende', 'energikild', 'energiforbrug', 'madspild', 'ressourceforbrug', 
             'kemikaliesikkerhed', 'vildtplej', 'naturplej', 'naturbeskytte', 'beskyt natur', 'naturbevar', 
             'bevare natur', 'naturudvikling', 'udvikle natur', 'økologi', 'økosystem', 'skovpleje', 'pleje af natur',
             'pleje af skov', 'pleje af vild', 'biotoppleje', 'træpleje', 'ressourcebespar', 'ressourcebevid',
             'ressourceanvende', 'materialespild', 'ressourcespild', 'spild af materiale', 'spild af ressource', 'repar']

In [ ]:
stopword_list = {
    "arbejdsmiljø", "bejdsmiljø", "netværksmiljø", "kontormiljø", "bomiljøer", 
    "staldmiljø", "værksmiljø", "gågademiljø", "læringsmiljø", "grøntsager", 
    "grønsag", "pyntegrønt", "klimasystemer", "klimaanlæg", "indeklima", "miljøministeriets", "arbejds miljø"
    "klima og ventilationsanlæg", "klimaog ventilationsanlæg"
}

### 2.1 Purpose 

In [ ]:
def clean_text(text):
    # Remove hyphen followed by a line break and join the split word
    # This regex looks for hyphens followed by a line break or space and joins the words
    text = re.sub(r'-\s*\n\s*', '', text)  # Handles hyphenated line breaks
    
    # Remove other punctuation and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)

    # Use spaCy to process and lemmatize the text
    doc = nlp(text)

    # Lemmatize each token while preserving case of acronyms
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    return lemmatized_text


In [ ]:
df_e2015['cleaned_formål_og_opdeling'] = df_e2015['formål_og_opdeling'].apply(clean_text)


In [ ]:
def find_matches(text):
    matches = []
    
    # First, check for multi-word terms directly in the text (without splitting)
    for word in word_list:
        # Handle multi-word terms (case-insensitive match)
        if " " in word:  # This checks if the word in the list is a phrase
            if word.lower() in text.lower():  # Check for multi-word phrases in the text
                matches.append(word)
    
    # Then, split the text into words for more granular matching for single words
    words_in_text = text.split()
    
    for word in word_list:
        if " " not in word:  # This only handles single-word terms
            # Check each word in the text
            for text_word in words_in_text:
                # Check if the stopword is a part of the current word in the text
                if not any(stopword in text_word.lower() for stopword in stopword_list):
                    # For acronyms like "EPD", use case-sensitive search; for others, search case-insensitively
                    if word.isupper() and word in text_word:  # Case-sensitive search for acronyms
                        matches.append(word)
                    elif word.lower() in text_word.lower():  # Case-insensitive search for non-acronyms
                        matches.append(word)

    return matches


In [ ]:
df_e2015['grøn_formål_og_opdeling'] = df_e2015['cleaned_formål_og_opdeling'].apply(find_matches)


### 2.2. Competencies prior to admission 

In [ ]:
def clean_text(text):
    # Remove hyphen followed by a line break and join the split word
    # This regex looks for hyphens followed by a line break or space and joins the words
    text = re.sub(r'-\s*\n\s*', '', text)  # Handles hyphenated line breaks
    
    # Remove other punctuation and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)

    # Use spaCy to process and lemmatize the text
    doc = nlp(text)

    # Lemmatize each token while preserving case of acronyms
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    return lemmatized_text

In [ ]:
df_e2015['cleaned_kompetencer_forud_optagelse'] = df_e2015['kompetencer_forud_optagelse'].apply(clean_text)


In [ ]:
def find_matches(text):
    matches = []
    
    # First, check for multi-word terms directly in the text (without splitting)
    for word in word_list:
        # Handle multi-word terms (case-insensitive match)
        if " " in word:  # This checks if the word in the list is a phrase
            if word.lower() in text.lower():  # Check for multi-word phrases in the text
                matches.append(word)
    
    # Then, split the text into words for more granular matching for single words
    words_in_text = text.split()
    
    for word in word_list:
        if " " not in word:  # This only handles single-word terms
            # Check each word in the text
            for text_word in words_in_text:
                # Check if the stopword is a part of the current word in the text
                if not any(stopword in text_word.lower() for stopword in stopword_list):
                    # For acronyms like "EPD", use case-sensitive search; for others, search case-insensitively
                    if word.isupper() and word in text_word:  # Case-sensitive search for acronyms
                        matches.append(word)
                    elif word.lower() in text_word.lower():  # Case-insensitive search for non-acronyms
                        matches.append(word)

    return matches

In [ ]:
df_e2015['grøn_kompetencer_forud_optagelse'] = df_e2015['cleaned_kompetencer_forud_optagelse'].apply(find_matches)


### 2.3. Competencies in the main course

In [ ]:
def clean_text(text):
    # Remove hyphen followed by a line break and join the split word
    # This regex looks for hyphens followed by a line break or space and joins the words
    text = re.sub(r'-\s*\n\s*', '', text)  # Handles hyphenated line breaks
    
    # Remove punctuation and non-alphanumeric characters, except parentheses
    text = re.sub(r'[^\w\s\(\)]', '', text)

    # Use spaCy to process and lemmatize the text
    doc = nlp(text)

    # Lemmatize each token while preserving case of acronyms
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    return lemmatized_text

In [ ]:
df_e2015['cleaned_kompetencer_hovedforløb'] = df_e2015['kompetencer_hovedforløb'].apply(clean_text)


In [ ]:
def find_matches(text):
    matches = []
    
    # First, check for multi-word terms directly in the text (without splitting)
    for word in word_list:
        # Handle multi-word terms (case-insensitive match)
        if " " in word:  # This checks if the word in the list is a phrase
            if word.lower() in text.lower():  # Check for multi-word phrases in the text
                matches.append(word)
    
    # Then, split the text into words for more granular matching for single words
    words_in_text = text.split()
    
    for word in word_list:
        if " " not in word:  # This only handles single-word terms
            # Check each word in the text
            for text_word in words_in_text:
                # Check if the stopword is a part of the current word in the text
                if not any(stopword in text_word.lower() for stopword in stopword_list):
                    # For acronyms like "EPD", use case-sensitive search; for others, search case-insensitively
                    if word.isupper() and word in text_word:  # Case-sensitive search for acronyms
                        matches.append(word)
                    elif word.lower() in text_word.lower():  # Case-insensitive search for non-acronyms
                        matches.append(word)

    return matches

In [ ]:
df_e2015['grøn_kompetencer_hovedforløb'] = df_e2015['cleaned_kompetencer_hovedforløb'].apply(find_matches)


### 2.4. Final exam

In [ ]:
def clean_text(text):
    # Remove hyphen followed by a line break and join the split word
    # This regex looks for hyphens followed by a line break or space and joins the words
    text = re.sub(r'-\s*\n\s*', '', text)  # Handles hyphenated line breaks
    
    # Remove other punctuation and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)

    # Use spaCy to process and lemmatize the text
    doc = nlp(text)

    # Lemmatize each token while preserving case of acronyms
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    return lemmatized_text

In [ ]:
df_e2015['cleaned_afsluttende_prøve'] = df_e2015['afsluttende_prøve'].apply(clean_text)


In [ ]:
def find_matches(text):
    matches = []
    
    # First, check for multi-word terms directly in the text (without splitting)
    for word in word_list:
        # Handle multi-word terms (case-insensitive match)
        if " " in word:  # This checks if the word in the list is a phrase
            if word.lower() in text.lower():  # Check for multi-word phrases in the text
                matches.append(word)
    
    # Then, split the text into words for more granular matching for single words
    words_in_text = text.split()
    
    for word in word_list:
        if " " not in word:  # This only handles single-word terms
            # Check each word in the text
            for text_word in words_in_text:
                # Check if the stopword is a part of the current word in the text
                if not any(stopword in text_word.lower() for stopword in stopword_list):
                    # For acronyms like "EPD", use case-sensitive search; for others, search case-insensitively
                    if word.isupper() and word in text_word:  # Case-sensitive search for acronyms
                        matches.append(word)
                    elif word.lower() in text_word.lower():  # Case-insensitive search for non-acronyms
                        matches.append(word)

    return matches

In [ ]:
df_e2015['grøn_afsluttende_prøve'] = df_e2015['cleaned_afsluttende_prøve'].apply(find_matches)


In [ ]:
df_e2015.to_pickle('df_e2015_ny.pkl')


## 3. Add educational fields and date

In [ ]:
df_e2015.rename(columns={'Subfolder': 'uddannele'}, inplace=True)

In [ ]:
mapping_df = pd.read_excel('hovedområder.xlsx')

In [ ]:
mapping_df.columns = ['uddannele', 'hovedområde']

In [ ]:
df_e2015['uddannele'] = df_e2015['uddannele'].str.lower()
mapping_df['uddannele'] = mapping_df['uddannele'].str.lower()

In [ ]:
df_e2015 = df_e2015.merge(mapping_df, on='uddannele', how='left')


In [ ]:
df_e2015['Date'] = pd.to_datetime(df_e2015['Date'], format='%d/%m/%Y')


In [ ]:
df_e2015['year'] = df_e2015['Date'].dt.year

In [ ]:
df_e2015 = df_e2015.sort_values('Date').drop_duplicates(subset=['uddannele', 'year'], keep='last')

In [ ]:
def create_annual_timeline(df):
    # Define the timeline range from the first year to 2024
    min_year = df['year'].min()
    max_year = 2024  # Ensure the timeline goes up to 2024
    
    # Create a DataFrame with a row for each year within the range for each education
    all_years = pd.DataFrame({'year': range(min_year, max_year + 1)})
    
    # Merge the new years DataFrame with the existing data, keeping all years
    df = pd.merge(all_years, df, on='year', how='left')
    
    # Fill missing rows with data from the previous year (forward fill)
    df = df.ffill()
    
    return df


In [ ]:
df_e2015 = (
    df_e2015.groupby('uddannele', group_keys=False)
    .apply(create_annual_timeline)
)


In [ ]:
df_e2015 = df_e2015.sort_values(['uddannele', 'year']).reset_index(drop=True)

In [ ]:
excel_file = "DISCED gruppering af EUD.xlsx"
df_DISCED = pd.read_excel(excel_file)

In [ ]:
df_e2015 = df_e2015.merge(df_DISCED[['uddannele', 'Område_Omkodet']], on='uddannele', how='left')

In [ ]:
df_e2015.to_pickle('df_e2015_f_WE.pkl')

## 4. Divide documents by competencies

In [ ]:
df_new = pd.DataFrame(new_rows)

In [ ]:
def extract_skill_descriptions_formål(text):
    # Updated regex to capture digits followed by a parenthesis with or without a space, e.g., "8)" or "8 )"
    pattern = r'(\d{1,2}\s?\))\s*(.*?)(?=\d{1,2}\s?\)|stk \d|$)'
    
    # Extract skill descriptions based on the updated pattern
    matches = re.findall(pattern, text, re.DOTALL)
    
    if matches:
        # Extract just the descriptions (ignore the numbers and closing parentheses)
        skill_descriptions = [match[1].strip() for match in matches]
    else:
        # If no matches, take all text before "stk 2" as one skill
        if "stk 2" in text:
            skill_descriptions = [text.split("stk 2")[0].strip()]
        else:
            skill_descriptions = [text.strip()]  # If no "stk 2", take all text as one skill
    
    return skill_descriptions

# Generic extract_skill_descriptions function for other sections
def extract_skill_descriptions(text):
    # Regex to capture digits followed by a parenthesis with or without a space
    pattern = r'(\d{1,2}\s?\))\s*(.*?)(?=\d{1,2}\s?\)|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    skill_descriptions = [match[1].strip() for match in matches]
    return skill_descriptions

# Function for cleaned_formål_og_opdeling
def divide_formål_og_opdeling(text):
    if "stk 2" in text:
        text = text.split("stk 2")[0]
    return extract_skill_descriptions_formål(text)

# Function for cleaned_kompetencer_forud_optagelse with exclusions
exclude_keywords = ["aniveau", "bniveau", "cniveau", "dniveau", "eniveau", "fniveau", "gniveau"]

def divide_kompetencer_forud_optagelse(text):
    skill_descriptions = extract_skill_descriptions(text)
    filtered_skills = [
        skill for skill in skill_descriptions
        if not any(keyword in skill.lower() for keyword in exclude_keywords)
    ]
    return filtered_skills

# Function for cleaned_kompetencer_hovedforløb
def divide_kompetencer_hovedforløb(text):
    if "stk 2" in text:
        text = text.split("stk 2")[0]
    return extract_skill_descriptions(text)

# Initialize the list for new rows
new_rows = []

# Iterate over each row in the original DataFrame
for _, row in df_e2015.iterrows():
    # Extract and divide sections
    sections = {
        "cleaned_formål_og_opdeling": divide_formål_og_opdeling(row["cleaned_formål_og_opdeling"]),
        "cleaned_kompetencer_forud_optagelse": divide_kompetencer_forud_optagelse(row["cleaned_kompetencer_forud_optagelse"]),
        "cleaned_kompetencer_hovedforløb": divide_kompetencer_hovedforløb(row["cleaned_kompetencer_hovedforløb"]),
        "cleaned_afsluttende_prøve": [row["cleaned_afsluttende_prøve"]]  # One row per original
    }

    # Loop through each section and its extracted skills
    for section_name, skills in sections.items():
        for skill in skills:
            new_rows.append({
                "year": row["year"],
                "uddannele": row["uddannele"],
                "Date": row["Date"],
                "hovedområde": row["hovedområde"],
                "Område_Omkodet": row["Område_Omkodet"],
                "section": section_name,
                "skill_description": skill
            })

In [ ]:
df_new.head()

In [ ]:
nlp = spacy.load('da_core_news_sm')

In [ ]:
word_list = ['luftkvalitet', 'akvakultur', 'bionedbryd', 'biodivers', 'biobrændstof', 'biomasse', 
             'co2', 'cirkulær', 'ren energi', 'klima', 'bevarelse', 'katastroferisikostyring', 
             'energieffekt', 'energihandel', 'miljø', 'geotermisk', 'gletsjerforsk', 'grøn', 
             'hydrologi', 'arealanvend', 'landskabsarkitekt', 'LEED', 'leadership in energy and environmental design',
             'naturlige pesticider', 'naturforsk', 
             'foruren', 'genbrug', 'vedvarende', 'solenergi', 'håndtering af affald', 'behandling af affald', 'affaldshåndt',
             'affaldsbehandl', 'affaldssort', 'bæredygtig', 'affaldsredu', 'spildevand', 
             'spildevandsreduktion', 'behandling af spildevand', 'reduktion af spildevand', 'vandressource', 
             'vejrsikring', 'dyreliv', 'vindenergi', 'udled', 'drivgas', 'ressourceeffekt', 
             'reduktion af material', 'vandforbrug', 'regenerativ', 'genanvend', 
             'giftige kemikalie', 'giftige stof', 'EPD', 'livscyklus', 'LCA', 'vedligehold', 'energioptim', 
             'ressourceanvende', 'energikild', 'energiforbrug', 'madspild', 'ressourceforbrug', 
             'kemikaliesikkerhed', 'vildtplej', 'naturplej', 'naturbeskytte', 'beskyt natur', 'naturbevar', 
             'bevare natur', 'naturudvikling', 'udvikle natur', 'økologi', 'økosystem', 'skovpleje', 'pleje af natur',
             'pleje af skov', 'pleje af vild', 'biotoppleje', 'træpleje', 'ressourcebespar', 'ressourcebevid',
             'ressourceanvende', 'materialespild', 'ressourcespild', 'spild af materiale', 'spild af ressource', 'repar']

In [ ]:
stopword_list = {
    "arbejdsmiljø", "bejdsmiljø", "netværksmiljø", "kontormiljø", "bomiljøer", 
    "staldmiljø", "værksmiljø", "gågademiljø", "læringsmiljø", "grøntsager", 
    "grønsag", "pyntegrønt", "klimasystemer", "klimaanlæg", "indeklima", "miljøministeriets", "arbejds miljø"
    "klima og ventilationsanlæg", "klimaog ventilationsanlæg"
}

In [ ]:
def find_matches(text):
    matches = []
    
    # Lowercase text for case-insensitive matching
    text_lower = text.lower()
    words_in_text = text.split()

    # Check for multi-word terms in the text
    for word in word_list:
        if " " in word:  # Multi-word terms
            if word.lower() in text_lower:
                matches.append(word)
    
    # Check for single-word terms in the text
    for word in word_list:
        if " " not in word:  # Single-word terms only
            for text_word in words_in_text:
                if not any(stopword in text_word.lower() for stopword in stopword_list):
                    if word.isupper() and word in text_word:  # Acronyms: Case-sensitive
                        matches.append(word)
                    elif word.lower() in text_word.lower():  # Regular words: Case-insensitive
                        matches.append(word)
    
    return matches

In [ ]:
df_new['grøn_words'] = df_new['skill_description'].apply(find_matches)

In [ ]:
df_new['is_green'] = df_new['grøn_words'].apply(lambda x: bool(x))

## 5. Separate final exam from competencies

In [ ]:
df_new = df_new[df_new['section'] != "cleaned_afsluttende_prøve"].reset_index(drop=True)


In [ ]:
prøve = df_new[df_new['section'] == 'cleaned_afsluttende_prøve']


In [ ]:
prøve.rename(columns={'grøn_words': 'grøn_words_final'}, inplace=True)


In [ ]:
prøve['green_or_not'] = prøve['grøn_words_final'].apply(lambda x: False if not x else True)

In [ ]:
prøve.to_pickle('prøve.pkl')

In [ ]:
df_new.to_pickle('df_e2015_f_WE_3.pkl')

## 6. Classification and validation with LLM approach

In [ ]:
tokenizer = tiktoken.encoding_for_model("gpt-4")

In [ ]:
def calculate_tokens(row):
    prompt = format_prompt(row)
    tokens = tokenizer.encode(prompt)
    return len(tokens)

In [ ]:
df_new['token_count'] = df_new.apply(calculate_tokens, axis=1)

In [ ]:
input_token_limit = 3096

# Filter rows exceeding the limit
rows_exceeding_limit = df_new[df_new['token_count'] > input_token_limit]

# Display rows exceeding the limit
print(f"Number of rows exceeding token limit: {len(rows_exceeding_limit)}")
rows_exceeding_limit

In [ ]:
import openai


In [ ]:
from openai import OpenAI


In [ ]:
client = OpenAI(api_key='(OpenAI KEY insert here')

In [ ]:
error_count = 0
max_errors_to_print = 10


In [ ]:
def classify_skill(row):
    global error_count  # Use the global error counter

    # Perform dictionary-based classification
    is_green = len(row['grøn_words']) > 0  # Based on dictionary approach

    # Create the prompt for classification
    classification_prompt = f"""
    Skill description: "{row['skill_description']}"
    Year: "{row['year']}"
    Education: "{row['uddannele']}"
    Category: "{row['Område_Omkodet']}"
    Green words detected: "{', '.join(row['grøn_words']) if row['grøn_words'] else 'None'}"
    Question: Er denne færdighedsbeskrivelse relateret til grønne færdigheder? 
    Grønne færdigheder defineres bredt som personlige kvaliteter, færdigheder, viden, evner og aktiviteter, 
    der bidrager til bæredygtig udvikling ved 1) at reducere energiforbrug, 
    2) beskytte økosystemer eller biodiversitet eller 
    3) minimere affald eller emissioner. 
    Start dit svar med 'Ja' eller 'Nej'.
    """
    try:
        # Get the classification from GPT-4o
        classification_response = client.chat.completions.create(
            model="gpt-4o-mini",  # Specify the GPT-4o model
            messages=[
                {"role": "system", "content": "Du er en ekspert i miljømæssig bæredygtighed og uddannelse."},
                {"role": "user", "content": classification_prompt}
            ],
            temperature=0.5,  # Adjust for deterministic outputs
        )
        # Extract the model's classification
        classification_text = classification_response.choices[0].message.content.strip()
        is_green_model = classification_text.lower().startswith("ja")

        # Check if the classifications differ
        explanation = None
        if is_green_model != is_green:
            # Create the explanation prompt
            explanation_prompt = f"""
            Skill description: "{row['skill_description']}"
            Year: "{row['year']}"
            Education: "{row['uddannele']}"
            Category: "{row['Område_Omkodet']}"
            Green words detected: "{', '.join(row['grøn_words']) if row['grøn_words'] else 'None'}"
            Question: Forklar kort hvorfor denne færdighedsbeskrivelse er {('ikke ' if not is_green_model else '')}relateret til grønne færdigheder.
            """
            explanation_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Du er en ekspert i miljømæssig bæredygtighed og uddannelse."},
                    {"role": "user", "content": explanation_prompt}
                ],
                temperature=0.5,
            )
            # Extract the explanation
            explanation = explanation_response.choices[0].message.content.strip()

        return is_green_model, explanation

    except Exception as e:
        # Print error for the first 10 errors only
        if error_count < max_errors_to_print:
            print(f"Error processing row: {e}")
        error_count += 1
        return None, None

In [ ]:
df_new[['is_green_classification', 'chatgpt_response']] = df_new.apply(
    lambda row: pd.Series(classify_skill(row)), axis=1
)


In [ ]:
df_new.to_pickle('df_e2015_WE_.pkl')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

### 6.1 Validation 

In [ ]:
accuracy = accuracy_score(df_new['is_green'], df_new['is_green_classification'])

# Calculate the precision
precision = precision_score(df_new['is_green'], df_new['is_green_classification'])

# Calculate the sensitivity (recall)
sensitivity = recall_score(df_new['is_green'], df_new['is_green_classification'])


In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Sensitivity (Recall): {sensitivity}")
print(f"Precision: {precision}")


In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
tn, fp, fn, tp = confusion_matrix(df_new['is_green'], df_new['is_green_classification']).ravel()


In [ ]:
specificity = tn / (tn + fp)

# Print specificity
print(f"Specificity: {specificity}")


## 7. Validation of LLM approach with manual review

In [ ]:
updated_df = pd.read_excel('df_WE.xlsx')

In [ ]:
df_new = df_new.reset_index()
updated_df = updated_df.reset_index()


In [ ]:
filtered_df = updated_df[updated_df['final_assessment'] == 'x']

In [ ]:
filtered_df['chatgpt_response'] = filtered_df.apply(
    lambda row: 
    df_new.loc[
        (df_new['skill_description'] == row['skill_description']) &
        (df_new['section'] == row['section']) &
        (df_new['year'] == row['year']) &
        (df_new['uddannele'] == row['uddannele']),
        'chatgpt_response'
    ].values[0] if pd.isna(row['chatgpt_response']) or row['chatgpt_response'] == '' else row['chatgpt_response'],
    axis=1
)


In [ ]:
filtered_df.to_excel("filtered_with_updated_chatgpt_response.xlsx", index=False)

In [ ]:
df_new['final_assessment'] = df_new.apply(
    lambda row: 
    filtered_df.loc[
        (filtered_df['chatgpt_response'] == row['chatgpt_response']),
        'final_assessment'
    ].values[0] if row['chatgpt_response'] in filtered_df['chatgpt_response'].values else None,
    axis=1
)

In [ ]:
df_new['green_or_not'] = df_new.apply(
    lambda row: row['is_green_classification'] if row['final_assessment'] == 'x' else row['is_green'],
    axis=1
)


In [ ]:
df_new['grøn_words_final'] = df_new.apply(
    lambda row: [] if row['final_assessment'] == 'x' else row['grøn_words'],
    axis=1
)

## 8. Add final exams to dataset

In [ ]:
with open('prøve.pkl', 'rb') as file:
    prøve = pickle.load(file)

In [ ]:
df_new_final = pd.concat([df_new, prøve], ignore_index=True)


In [ ]:
df_new_final.to_pickle('df_e2015_WE_final.pkl')

## 9. Recode dataset - Rows are training ordinances instead of competencies

In [ ]:
educations = df_new_final.groupby(['uddannele', 'year'], as_index=False).agg({
    'Date': 'first',  # Keep the first date (or adjust logic as needed)
    'hovedområde': 'first',  # Keep the first hovedområde (or adjust logic)
    'Område_Omkodet': 'first',  # Keep the first Område_Omkodet (or adjust logic)
    'green_or_not': 'mean'  # Calculate the mean of 'green_or_not' for green_outcome
})


In [ ]:
educations.head(10)